# Trabalho 1

11218813 Bruna Magrini da Cruz<br>
11218809 Flavio Salles<br>
11219004 Yasmin Osajima de Araujo

# Bibliotecas

In [1]:
%pip install glfw
%pip install PyOpenGL



Note: you may need to restart the kernel to use updated packages.


In [2]:
import glfw
from OpenGL.GL import *
import OpenGL.GL.shaders
import numpy as np
import math


# Janela

In [3]:
glfw.init()
glfw.window_hint(glfw.VISIBLE, glfw.FALSE)
window = glfw.create_window(
    1020, 900, "Trabalho 1 - Jardim do ICMC", None, None)
glfw.make_context_current(window)


# GLSL

Aqui, utilizamos o GLSL para definir o Vertex Shader (processamento dos vértices individuais) e Fragment Shader (processamento das cores dos pixels).

In [4]:
# Recebe as duas coordenadas de um ponto e a matriz de transformação
vertex_code = """
        attribute vec2 position;
        uniform mat4 mat_transformation;
        void main(){
            gl_Position = mat_transformation * vec4(position,0.0,1.0);
        }
        """

# Recebe os dados de cor do programa
fragment_code = """
        uniform vec4 color;
        void main(){
            gl_FragColor = color;
        }
        """


# Compilação e Linkagem

Requisitando slot para a GPU para os programas *Vertex Shader* e *Fragment Shader*, associando o código-fonte aos slots solicitados e compilando ambos shaders.

In [5]:
program = glCreateProgram()
vertex = glCreateShader(GL_VERTEX_SHADER)
fragment = glCreateShader(GL_FRAGMENT_SHADER)

glShaderSource(vertex, vertex_code)
glShaderSource(fragment, fragment_code)

# Compilando o Vertex Shader
glCompileShader(vertex)
if not glGetShaderiv(vertex, GL_COMPILE_STATUS):
    error = glGetShaderInfoLog(vertex).decode()
    print(error)
    raise RuntimeError("Erro de compilacao do Vertex Shader")

# Compilando o Fragment Shader
glCompileShader(fragment)
if not glGetShaderiv(fragment, GL_COMPILE_STATUS):
    error = glGetShaderInfoLog(fragment).decode()
    print(error)
    raise RuntimeError("Erro de compilacao do Fragment Shader")


Em seguida, associando os programas compilados ao programa principal e linkando o programa.

In [6]:
glAttachShader(program, vertex)
glAttachShader(program, fragment)

# Compilando o programa
glLinkProgram(program)
if not glGetProgramiv(program, GL_LINK_STATUS):
    print(glGetProgramInfoLog(program))
    raise RuntimeError('Linking error')

# Definindo o programa como o default
glUseProgram(program)


# Definição de Elementos

Em seguida, são definidos os vértices da cada objeto: pá, canteiro e as 5 flores.

In [7]:
def get_vertices_circulo(x_center, y_center, num_vertices, radius):
    circulo = np.zeros(num_vertices, [("position", np.float32, 2)])
    pi = np.pi
    angle = 0.0
    for counter in range(num_vertices):
        angle += 2*pi/num_vertices
        x = math.cos(angle)*radius + x_center
        y = math.sin(angle)*radius + y_center
        circulo[counter] = [x, y]
    return circulo


def get_vertices_pa():
    pa = np.zeros(12, [("position", np.float32, 2)])
    pa['position'] = [
        (0.033, 0.5), (0.033, 0.513), (-0.033, 0.5), (-0.033, 0.513),
        (0.015, 0.1), (0.015, 0.5), (-0.015, 0.1), (-0.015, 0.5),
        (0.1, 0.1), (0.1, 0.0), (-0.1, 0.1), (-0.1, 0.0)
    ]
    pa = np.append(pa, get_vertices_circulo(0, 0, 32, 0.1))
    return pa


def get_vertices_flor4():
    flor = np.zeros(40, [("position", np.float32, 2)])
    flor['position'] = [
        # Pétala 1
        (0, 0), (0.3, -0.1), (0.4, 0), (0.3, 0.1), (0, 0),
        # Pétala 2
        (0, 0), (0.29, 0.15), (0.3, 0.3), (0.15, 0.29), (0, 0),
        # Pétala 3
        (0, 0), (0.1, 0.3), (0, 0.4), (-0.1, 0.3), (0, 0),
        # Pétala 4
        (0, 0), (-0.15, 0.29), (-0.3, 0.3), (-0.29, 0.15), (0, 0),
        # Pétala 5
        (0, 0), (-0.3, 0.1), (-0.4, 0), (-0.3, -0.1), (0, 0),
        # Pétala 6
        (0, 0), (-0.29, -0.15), (-0.3, -0.3), (-0.15, -0.29), (0, 0),
        # Pétala 7
        (0, 0), (-0.1, -0.3), (0, -0.4), (0.1, -0.3), (0, 0),
        # Pétala 8
        (0, 0), (0.15, -0.29), (0.3, -0.3), (0.29, -0.15), (0, 0),
    ]
    # Meio
    flor = np.append(flor, get_vertices_circulo(0, 0, 32, 0.2))
    return flor


In [8]:
# No total, 154 pontos serão definidos, cada um usando 2 coordenadas
vertices = np.zeros(154, [("position", np.float32, 2)])

# Todos os pontos são definidos/adicionados na variável de vertices
vertices['position'] = [
    # -------------- Flor 1 (0 - 21) --------------#
    # Meio
    (-0.01, +0.1), (+0.08, +0.02), (-0.08, 0), (+0.02, -0.07),
    # Petala 1
    (-0.01, +0.1), (+0.16, +0.29), (-0.13, +0.32),
    # Petala 2
    (+0.28, +0.29), (+0.04, +0.05), (+0.31, +0.07),
    # Petala 3
    (+0.33, -0.06), (+0.06, -0.03), (+0.2, -0.28),
    # Petala 4
    (+0.02, -0.07), (-0.07, -0.24), (+0.13, -0.31),
    # Petala 5
    (-0.05, -0.03), (-0.2, -0.25), (-0.34, -0.07),
    # Petala 6
    (-0.36, +0.06), (-0.05, +0.05), (-0.2, +0.2),

    # -------------- Flor 2 (22 - 70) --------------#
    # Meio
    (0, +0.1), (+0.08, +0.05), (+0.08, -0.04), (0, -0.09),
    (-0.08, -0.04), (-0.08, +0.05), (0, +0.1),
    # Petala 1
    (0, +0.1), (0, +0.19), (+0.08, +0.24), (+0.16, +0.19),
    (+0.16, +0.1), (+0.08, +0.05), (0, +0.1),
    # Petala 2
    (+0.08, +0.05), (+0.16, +0.1), (+0.24, +0.06), (+0.24, -0.04),
    (+0.16, -0.09), (+0.08, -0.04), (+0.08, +0.05),
    # Petala 3
    (+0.08, -0.04), (+0.16, -0.09), (+0.16, -0.18), (+0.08, -0.23),
    (0, -0.18), (0, -0.09), (+0.08, -0.04),
    # Petala 4
    (0, -0.09), (0, -0.18), (-0.08, -0.22), (-0.16, -0.18),
    (-0.16, -0.09), (-0.08, -0.04), (0, -0.09),
    # Petala 5
    (-0.08, -0.04), (-0.16, -0.09), (-0.24, -0.04), (-0.24, +0.05),
    (-0.16, +0.1), (-0.08, +0.05), (-0.08, -0.04),
    # Petala 6
    (-0.08, +0.05), (-0.16, +0.1), (-0.16, +0.19), (-0.08, +0.24),
    (0, +0.19), (0, +0.1), (-0.08, +0.05),

    # -------------- Flor 3 (71 - 113) --------------#
    # Meio
    (0, +0.1), (+0.07, +0.06), (+0.07, -0.02), (0, -0.06),
    (-0.07, -0.02), (-0.07, +0.06), (0, +0.1),
    # Petala 1
    (+0.01, +0.12), (+0.03, +0.2), (+0.11, +0.2),
    (+0.14, +0.13), (+0.08, 0.08), (+0.01, +0.12),
    # Petala 2
    (+0.09, +0.06), (+0.17, +0.09), (+0.21, +0.02),
    (+0.17, -0.04), (+0.09, -0.02), (+0.09, +0.06),
    # Petala 3
    (+0.09, -0.04), (+0.15, -0.09), (+0.12, -0.17),
    (+0.03, -0.16), (+0.01, -0.08), (+0.09, -0.04),
    # Petala 4
    (-0.01, -0.08), (-0.03, -0.17), (-0.11, -0.17),
    (-0.15, -0.1), (-0.08, -0.04), (-0.01, -0.08),
    # Petala 5
    (-0.09, -0.02), (-0.18, -0.05), (-0.22, +0.02),
    (-0.17, +0.09), (-0.09, +0.06), (-0.09, -0.02),
    # Petala 6
    (-0.09, +0.07), (-0.15, +0.13), (-0.11, +0.21),
    (-0.03, +0.2), (-0.01, 0.11), (-0.09, +0.07),

    # -------------- Canteiro (114 - 153) --------------#
    # Borda
    (-0.63, -0.63), (-0.63, 0.63), (0.63, -0.63), (0.63, 0.63),
    # Fundo
    (-0.6, -0.6), (-0.6, 0.6), (0.6, -0.6), (0.6, 0.6),
    # Padrão - 8 quadrados
    (-0.6, -0.6), (-0.6, -0.3), (-0.3, -0.6), (-0.3, -0.3),
    (-0.6, 0.0), (-0.6, 0.3), (-0.3, 0.0), (-0.3, 0.3),
    (-0.3, -0.3), (-0.3, 0.0), (0.0, -0.3), (0.0, 0.0),
    (-0.3, 0.3), (-0.3, 0.6), (0.0, 0.3), (0.0, 0.6),
    (0.0, -0.6), (0.0, -0.3), (0.3, -0.6), (0.3, -0.3),
    (0.0, 0.0), (0.0, 0.3), (0.3, 0.0), (0.3, 0.3),
    (0.3, -0.3), (0.3, 0.0), (0.6, -0.3), (0.6, 0.0),
    (0.3, 0.3), (0.3, 0.6), (0.6, 0.3), (0.6, 0.6),

]

# -------------- Flor 4 (154-313) --------------------#
vertices = np.append(vertices, get_vertices_circulo(0.1, 0.1, 32, 0.1))
vertices = np.append(vertices, get_vertices_circulo(0.1, -0.1, 32, 0.1))
vertices = np.append(vertices, get_vertices_circulo(-0.1, -0.1, 32, 0.1))
vertices = np.append(vertices, get_vertices_circulo(-0.1, 0.1, 32, 0.1))
vertices = np.append(vertices, get_vertices_circulo(0, 0, 32, 0.1))

# -------------- Pá (314 - 357) --------------------#
vertices = np.append(vertices, get_vertices_pa())

# -------------- Flor 5 (358 - 429) --------------------#
vertices = np.append(vertices, get_vertices_flor4())


In [9]:
# Variáveis que definem o inicio de cada objeto no vetor de vértices
OFFSET_FLOR_1 = 0
OFFSET_FLOR_2 = 22
OFFSET_FLOR_3 = 71
OFFSET_CANTEIRO = 114
OFFSET_FLOR_4 = 154
OFFSET_PA = 314
OFFSET_FLOR_5 = 358


# Transmitindo dados da CPU para a GPU

Nesse momento, o *Vertex* e *Fragment Shaders* estão compilados para que a GPU possa processá-los.
Por outro lado, as informações de vértices geralmente estão na CPU e devem ser transmitidas para a GPU.

Requisita um slot, envia o contéudo da variável ```vertices```  e associa as variáveis do programa GLSL (os shaders) com os dados.

In [10]:
# Requisita um buffer na GPU
buffer = glGenBuffers(1)
glBindBuffer(GL_ARRAY_BUFFER, buffer)

# Envia conteúdo da variável para o buffer
glBufferData(GL_ARRAY_BUFFER, vertices.nbytes, vertices, GL_DYNAMIC_DRAW)
glBindBuffer(GL_ARRAY_BUFFER, buffer)

stride = vertices.strides[0]  # Byte Inicial
offset = ctypes.c_void_p(0)  # Offset dos dados

# A variável loc contém a localização da variável position (que guarda as coordenadas dos vértices)
loc = glGetAttribLocation(program, "position")
# Definindo a variavel no Vertex Shader
glEnableVertexAttribArray(loc)

# Indicamos a GPU onde está o conteúdo (via posições stride/offsite) para a variável position
# Também definimos que são duas coordenadas, o tipo (GL_FLOAT) e que não deve ser normalizada (False)
glVertexAttribPointer(loc, 2, GL_FLOAT, False, stride, offset)


Pegando a localização da variável de cor para que possa ser alterado.

In [11]:
loc_color = glGetUniformLocation(program, "color")


# Eventos do Teclado

Abaixo é definido as variáveis de controle do jogo.

In [12]:
# Armazena o offset tx e ty de cada flor
offset_translacao = np.array([[0, 0, 0, 0, 0],
                              [0, 0, 0, 0, 0]], dtype=float)
# Armazena o fator de escala sx e sy de cada flor
fator_escala = np.array([[0.4, 0.5, 0.5, 0.5, 0.3],
                         [0.4, 0.5, 0.5, 0.5, 0.3]], dtype=float)
# Armazena o ângulo de rotação de cada flor
theta_rotacao = np.array([0, 0, 0, 0, 0], dtype=float)

# Para cada flor, identifica se ela já foi plantada pelo usuário
flor_plantada = np.array([False, False, False, False, False])

# Indica qual a flor que o usuário tem selecionada
flor_atual = 0

# Informa se as 5 flores foram plantadas
fim_de_jogo = False


Em seguida, é definido os comandos que o usuário pode realizar e seus efeitos para o estado do jogo.

In [13]:
# Distância que as flores devem manter da borda do canteiro
MARGEM_BORDA = 0.15


def key_event(window, key, scancode, action, mods):
    global offset_translacao, fator_escala, theta_rotacao, flor_atual, flor_plantada
    margem_borda = 0.15

    # Transladar com as setas
    if key == 265 and not flor_plantada[flor_atual]:  # Cima
        if offset_translacao[1][flor_atual] + 0.01 < vertices[115][0][1] - MARGEM_BORDA:
            offset_translacao[1][flor_atual] += 0.01
    if key == 264 and not flor_plantada[flor_atual]:  # Baixo
        if offset_translacao[1][flor_atual] - 0.01 > vertices[114][0][1] + MARGEM_BORDA:
            offset_translacao[1][flor_atual] -= 0.01
    if key == 263 and not flor_plantada[flor_atual]:  # Esquerda
        if offset_translacao[0][flor_atual] - 0.01 > vertices[114][0][0] + MARGEM_BORDA:
            offset_translacao[0][flor_atual] -= 0.01
    if key == 262 and not flor_plantada[flor_atual]:  # Direita
        if offset_translacao[0][flor_atual] + 0.01 < vertices[115][0][1] - MARGEM_BORDA:
            offset_translacao[0][flor_atual] += 0.01

    # Escalar com as teclas + e -
    # Aumenta
    if (key == 334 or key == 61) and fator_escala[0][flor_atual] < 1 and not flor_plantada[flor_atual]:
        fator_escala[0][flor_atual] += 0.01
        fator_escala[1][flor_atual] += 0.01
    # Diminui
    if (key == 333 or key == 45) and fator_escala[0][flor_atual] > 0.15 and not flor_plantada[flor_atual]:
        fator_escala[0][flor_atual] -= 0.01
        fator_escala[1][flor_atual] -= 0.01

    # Rotacionar com as teclas < e >
    if key == 44 and not flor_plantada[flor_atual]:  # Anti-horário
        theta_rotacao[flor_atual] += 0.1
    if key == 46 and not flor_plantada[flor_atual]:  # Horário
        theta_rotacao[flor_atual] -= 0.1

    # Alterar entre as flores, utilizando as teclas de 1 até 4
    if key == 49 or key == 321:
        flor_atual = 0
    elif key == 50 or key == 322:
        flor_atual = 1
    elif key == 51 or key == 323:
        flor_atual = 2
    elif key == 52 or key == 324:
        flor_atual = 3
    elif key == 53 or key == 325:
        flor_atual = 4

    # Fixar flores na tela
    if key == 257:  # Enter
        flor_plantada[flor_atual] = True


glfw.set_key_callback(window, key_event)


Definindo uma função genérica para gerar a matriz de transformação das flores de acordo com as variáveis modificadas pelos eventos do teclado:

In [14]:
def cria_matriz_transformacao(index=flor_atual, transladar=True, rotacionar=True, escalar=True):
    t_x = offset_translacao[0][index] if transladar else 1
    t_y = offset_translacao[1][index] if transladar else 1
    s_x = fator_escala[0][index] if escalar else 1
    s_y = fator_escala[1][index] if escalar else 1
    th = theta_rotacao[index] if rotacionar else 1

    return np.array([math.cos(th)*s_x, -math.sin(th)*s_y, 0.0, t_x,
                     math.sin(th)*s_x, math.cos(th)*s_y, 0.0, t_y,
                     0.0, 0.0, 1.0, 0.0,
                     0.0, 0.0, 0.0, 1.0], np.float32)


# Execução

Exibindo a janela.

In [15]:
glfw.show_window(window)


### Objetos

Aqui é definido como cada objeto deve ser desenhado, utilizando a matriz de transformação correspondente e as cores desejadas.

In [16]:
def desenhar_canteiro(program):
    matriz_canteiro = np.array([1.0, 0.0, 0.0, 0.0,
                                0.0, 1.0, 0.0, 0.0,
                                0.0, 0.0, 1.0, 0.0,
                                0.0, 0.0, 0.0, 1.0], np.float32)
    loc = glGetUniformLocation(program, "mat_transformation")
    glUniformMatrix4fv(loc, 1, GL_TRUE, matriz_canteiro)

    offset = OFFSET_CANTEIRO
    # Borda
    glUniform4f(loc_color, 0.20, 0.11, 0.03, 1.0)
    glDrawArrays(GL_TRIANGLE_STRIP, offset, 4)
    offset += 4
    # Fundo
    glUniform4f(loc_color, 0.38, 0.22, 0.07, 1.0)
    glDrawArrays(GL_TRIANGLE_STRIP, offset, 4)
    offset += 4
    # Padrao
    for i in range(8):
        glUniform4f(loc_color, 0.38, 0.26, 0.07, 1.0)
        glDrawArrays(GL_TRIANGLE_STRIP, offset, 4)
        offset += 4


def desenhar_flor0():
    offset = OFFSET_FLOR_1
    # Meio (Amarelo)
    glUniform4f(loc_color, 1, 1, 0, 1.0)
    glDrawArrays(GL_TRIANGLE_STRIP, offset, 4)
    offset += 4

    # Petalas (Brancas)
    for i in range(6):
        glUniform4f(loc_color, 1, 1, 1, 1.0)
        glDrawArrays(GL_TRIANGLE_STRIP, offset, 3)
        offset += 3


def desenhar_flor1():
    offset = OFFSET_FLOR_2
    glUniform4f(loc_color, 0.41, 0.35, 0.80, 1.0)
    glDrawArrays(GL_TRIANGLE_FAN, offset, 7)
    offset += 7

    for i in range(3):
        glUniform4f(loc_color, 0.28, 0.24, 0.54, 1.0)
        glDrawArrays(GL_TRIANGLE_FAN, offset, 7)
        offset += 7
        glUniform4f(loc_color, 1, 1, 1, 1.0)
        glDrawArrays(GL_TRIANGLE_FAN, offset, 7)
        offset += 7


def desenhar_flor2():
    offset = OFFSET_FLOR_3
    glUniform4f(loc_color, 0.87, 0.19, 0.39, 1.0)
    glDrawArrays(GL_TRIANGLE_FAN, offset, 7)
    offset += 7

    for i in range(6):
        glUniform4f(loc_color, 0.87, 0.19, 0.39, 1.0)
        glDrawArrays(GL_TRIANGLE_FAN, offset, 6)
        offset += 6


def desenhar_flor3():
    offset = OFFSET_FLOR_4
    for i in range(4):
        glUniform4f(loc_color, 0.87, 0.87, 0.0, 1.0)
        glDrawArrays(GL_TRIANGLE_FAN, offset, 32)
        offset += 32

    glUniform4f(loc_color, 0.9, 0.9, 0.9, 1.0)
    glDrawArrays(GL_TRIANGLE_FAN, offset, 32)


def desenhar_flor4():
    offset = OFFSET_FLOR_5
    for i in range(8):
        glUniform4f(loc_color, 1.0, 0.7, 0.1, 1.0)
        glDrawArrays(GL_TRIANGLE_STRIP, offset, 5)
        offset += 5

    glUniform4f(loc_color, 0.25, 0.12, 0.0, 1.0)
    glDrawArrays(GL_TRIANGLE_FAN, offset, 32)


def desenhar_flor(program, flor):
    loc = glGetUniformLocation(program, "mat_transformation")
    matriz_flor = cria_matriz_transformacao(flor)
    glUniformMatrix4fv(loc, 1, GL_TRUE, matriz_flor)

    if flor == 0:
        desenhar_flor0()
    elif flor == 1:
        desenhar_flor1()
    elif flor == 2:
        desenhar_flor2()
    elif flor == 3:
        desenhar_flor3()
    elif flor == 4:
        desenhar_flor4()


def desenhar_pa(program, flor=None, fim_de_jogo=False):
    loc = glGetUniformLocation(program, "mat_transformation")
    if fim_de_jogo:
        matriz_pa = np.array([1.0, 0.0, 0.0, 0.8,
                              0.0, 1.0, 0.0, -0.2,
                              0.0, 0.0, 1.0, 0.0,
                              0.0, 0.0, 0.0, 1.0], np.float32)
        glUniformMatrix4fv(loc, 1, GL_TRUE, matriz_pa)
    else:
        matriz_pa = cria_matriz_transformacao(flor, escalar=False)
        glUniformMatrix4fv(loc, 1, GL_TRUE, matriz_pa)

    offset = OFFSET_PA
    # Cabo da pá
    for i in range(2):
        glUniform4f(loc_color, 0.9, 0.9, 0.9, 1.0)
        glDrawArrays(GL_TRIANGLE_STRIP, offset, 4)
        offset += 4
    # Pá
    glUniform4f(loc_color, 0.4, 0.4, 0.4, 1.0)
    glDrawArrays(GL_TRIANGLE_STRIP, offset, 4)
    offset += 4
    glUniform4f(loc_color, 0.4, 0.4, 0.4, 1.0)
    glDrawArrays(GL_TRIANGLE_FAN, offset, 32)
    offset += 32


### Loop principal da janela.

In [17]:
print("Boas vindas ao Jardim do ICMC!")
print("------------------------------")
print("Como jardineiro, você deve plantar todas as 5 flores, utilizando as teclas de 1 a 5 para selecionar uma flor e ENTER para plantar.")
print("Para mover utilize as setas do teclado.")
print("Para escalar utilize as seguintes teclas: '+', '-'.")
print("Para rotacionar utilize as seguintes teclas: '<', '>'.")
print("Ao plantar todas as flores, você terá ajudado na construção do Jardim do ICMC.")
print("------------------------------")

while not glfw.window_should_close(window):
    glfw.poll_events()

    # Definindo a cor de fundo
    glClearColor(0.45, 0.59, 0.23, 1.0)
    glClear(GL_COLOR_BUFFER_BIT)

    desenhar_canteiro(program)

    def desenhar_plantando_flor(flor):
        # A flor vai ser desenhada na tela caso ela seja a flor selecionada pelo usuario (flor atual) ou caso ela já tenha sido plantada
        if flor_atual == flor or flor_plantada[flor]:
            # Caso a flor seja a atual e ainda não tenha sido plantada, a pá deve seguir ela
            if flor_atual == flor and not flor_plantada[flor]:
                desenhar_pa(program, flor=flor)
            desenhar_flor(program, flor)

    desenhar_plantando_flor(0)
    desenhar_plantando_flor(1)
    desenhar_plantando_flor(2)
    desenhar_plantando_flor(3)
    desenhar_plantando_flor(4)

    if np.sum(flor_plantada) == 5 and fim_de_jogo == False:
        print("Parabéns!!! Que lindo canteiro ^^")
        fim_de_jogo = True

    if fim_de_jogo:
        desenhar_pa(program, fim_de_jogo=True)

    glfw.swap_buffers(window)

glfw.terminate()


Boas vindas ao Jardim do ICMC!
------------------------------
Como jardineiro, você deve plantar todas as 5 flores, utilizando as teclas de 1 a 5 para selecionar uma flor e ENTER para plantar.
Para mover utilize as setas do teclado.
Para escalar utilize as seguintes teclas: '+', '-'.
Para rotacionar utilize as seguintes teclas: '<', '>'.
Ao plantar todas as flores, você terá ajudado na construção do Jardim do ICMC.
------------------------------
Parabéns!!! Que lindo canteiro ^^
